In [ ]:
# 序列的修改、散列和切片

# 在面向对象编程中，协议是非正式的接口，只在文档中定义，在代码中不定义。协议是非正式的，没有强制力。
# 比如序列协议值只需要__len__和__getitem__两个方法，任何类实现了这两个方法，就能用在任何期待序列的地方。该类是不是某个类的子类无关紧要。

# 切片原理

class MySeq:
    def __getitem__(self, index):
        print(index)
        return index

s = MySeq()
s[1]
s[1:4] # index 为slice对象
s[1:4:2]
s[1:4:2, 9] # index 为slice对象组成的元组
s[1:4:2, 7:9]

print(dir(slice))  # slice是内置类型，含有indices方法
# indices方法开放了内置序列实现的棘手逻辑，用于优雅地处理缺失索引和复数索引，以及长度超过目标序列的切片。
help(slice.indices) # 给定长度为len序列，计算扩展切片的起始start、结尾stop以及步幅stride
# 对长度为5的序列 
# slice(None, 10, 2).indices(5) 返回结果 (0, 5, 2)
# slice(-3, None, None).indices(5) 返回结果 (2, 5, 1) 

In [ ]:
# N维向量类
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools  # <1>


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components) # 采用"受保护的“实例属性，约定一般属性不用下划线开头_

    def __iter__(self):
        return iter(self._components) # 利用iter()函数构建一个迭代器

    def __repr__(self):
        components = reprlib.repr(self._components) # 利用reprlib.repr获取有限长度表示形式（中间含省略号...）
        components = components[components.find('['):-1] # 去掉 array('d', [0.0, ...])中的 array('d'和后面的)
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components)) # 直接构建bytes对象

    def __eq__(self, other): # 实现可散列协议
        return (len(self) == len(other) and # 长度检查非常必要，否则zip处理不同长度的数据会立即停止且不发出警告
                all(a == b for a, b in zip(self, other))) # 利用zip函数生成一个元组构成的生成器

    def __hash__(self): # 实现可散列协议
        hashes = (hash(x) for x in self)  # 计算各元素散列值并组成元组
        return functools.reduce(operator.xor, hashes, 0) # reduce和operator.xor计算聚合值，0为初值。高效易懂。

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self)) # 求模

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components) # 实现序列协议

    def __getitem__(self, index): # 实现序列协议
        cls = type(self)  # 目的是让切片结果也是Vector类对象
        if isinstance(index, slice): # 如果是slice则是切片
            return cls(self._components[index]) # 用array切片构造Vector类对象
        elif isinstance(index, numbers.Integral): # 如果是整数
            return self._components[index] # 返回元素
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls)) # 否则抛出异常。与Python内置的异常一致

    shortcut_names = 'xyzt' # 类属性

    def __getattr__(self, name):  # 解释器查找对象属性，而后类属性，接着沿继承数查找，最后调用__get_attr__方法
        cls = type(self) # 获取类名称
        if len(name) == 1: # 只处理单个字符属性
            pos = cls.shortcut_names.find(name) # 定位位置，返回相应元素
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name)) # 其他情况均抛出异常，与Python内置的异常一致
    
    def __set_attr__(self, name, value): # 为避免矛盾（属性可访问，似乎可以赋值，却会新建对象属性）必须实现属性赋值方法
        cls = type(self)   # 获取类名称
        if len(name) == 1: # 与获取属性一致，只处理单个字符属性
            if name in cls.shortcut_names: # 按各种情况处理属性赋值的异常提示信息
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attribute 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeErrors(msg)
        super().__setattr__(name, value) # 其他情况，调用父类的属性赋值

    def angle(self, n):  # <2>
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):  # <3>
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # N维超球面坐标hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())  # chain函数生成生成器表达式，无缝迭代向量的模和各个角坐标
            outer_fmt = '<{}>'  # 超球面坐标
        else:
            coords = self
            outer_fmt = '({})'  # 笛卡尔坐标
        components = (format(c, fmt_spec) for c in coords)  # 创建生成器表达式
        return outer_fmt.format(', '.join(components))  # 把以逗号分隔的格式化分量插入括号中。

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 直接把memoryview传给构造方法（构造方法用的是迭代对象了）

print(Vector([3.1, 4.2]))
print(Vector((3, 4, 5)))
print(Vector(range(10)))


v1 = Vector([3, 4, 5])
print(v1.x, v1.y)
x, y, z = v1
print(x, y, z)
v1_clone = eval(repr(v1))
print(v1 == v1_clone)
print(v1)
octets = bytes(v1)
print(octets)
print(abs(v1))
print(bool(v1), bool(Vector([0, 0])))

v2_clone = Vector.frombytes(bytes(v1))
print(v1_clone == v2_clone)
print(v2_clone)
print(format(v1))
print(format(v1, '.2f'))
print(format(v1, '.3e'))
print(format(v1, '0.5fh'))
print(hash(v1_clone), hash(v2_clone))
print(len([v1_clone, v2_clone]))
print(v1.x, v1.y)
print(v1[0], v1[len(v1)-1], v1[-1], v1[1:4])
